In [1]:
import pandas as pd
import sqlalchemy as sa
import urllib

params = urllib.parse.quote_plus('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=localhost;'
                      'Database=InteractDb_rc2;'
                      'Trusted_Connection=yes;')

engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
cnxn = engine.connect()

In [2]:
disease_id_df = pd.read_sql(sa.text('SELECT $node_id, disgenet_id AS dst FROM disease_nodes'), cnxn)
disease_id_df

,$node_id_9F94EAB3BA4046F396391C955D314E4B,dst
0,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C0018923
1,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C0018923
2,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C0018923
3,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C0018923
4,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C0018923
...,...,...
413277,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C0020681
413278,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C0020681
413279,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C1262019
413280,"{""type"":""node"",""schema"":""dbo"",""table"":""disease...",C1262019


In [3]:
df_drug_ids = pd.read_sql(sa.text('SELECT $node_id, drugbank_id FROM drug_nodes'), cnxn)
df_drug_ids

,$node_id_C59E823CFB2442BFB04D7FE55B42561F,drugbank_id
0,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00001
1,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00002
2,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00003
3,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00004
4,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00005
...,...,...
15230,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17382
15231,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17383
15232,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17384
15233,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17385


In [4]:
df = pd.read_csv("../parser/temp/drug_disease_edges.csv", sep="\t")
df

,drugbank-id,disgenet_id,name,assoc_cond,icd11Code,icd11Title,icd10Code,icd10Title
0,DB00305,C0017601,Glaucoma,Ab externo surgery Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
1,DB01085,C0017601,Glaucoma,Chronic open-angle glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
2,DB01234,C0017601,Glaucoma,Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
3,DB13316,C0017601,Glaucoma,Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
4,DB15071,C0017601,Glaucoma,Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
...,...,...,...,...,...,...,...,...
47978,DB01390,C0043515,Zollinger-Ellison syndrome,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin
47979,DB01390,C0043515,Zollinger-Ellison syndrome,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin
47980,DB01390,C0043515,Zollinger-Ellison syndrome,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin
47981,DB01390,C0232567,Hypergastrinemia,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin


In [5]:
df_ = pd.merge(left=df, right=df_drug_ids, left_on="drugbank-id", right_on="drugbank_id", how="left")
df_ = df_[['$node_id_C59E823CFB2442BFB04D7FE55B42561F', 'drugbank_id','disgenet_id', 'name', 'assoc_cond', 'icd11Code',
       'icd11Title', 'icd10Code', 'icd10Title',]]
df_.columns = ['from_id', 'src','disgenet_id', 'name', 'assoc_cond', 'icd11Code',
       'icd11Title', 'icd10Code', 'icd10Title',]
df_

,from_id,src,disgenet_id,name,assoc_cond,icd11Code,icd11Title,icd10Code,icd10Title
0,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00305,C0017601,Glaucoma,Ab externo surgery Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
1,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB01085,C0017601,Glaucoma,Chronic open-angle glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
2,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB01234,C0017601,Glaucoma,Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
3,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB13316,C0017601,Glaucoma,Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
4,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB15071,C0017601,Glaucoma,Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
...,...,...,...,...,...,...,...,...,...
47978,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB01390,C0043515,Zollinger-Ellison syndrome,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin
47979,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB01390,C0043515,Zollinger-Ellison syndrome,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin
47980,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB01390,C0043515,Zollinger-Ellison syndrome,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin
47981,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB01390,C0232567,Hypergastrinemia,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin


In [6]:
df__ = pd.merge(left=df_, right=disease_id_df, left_on="disgenet_id", right_on="dst", how="left")
df__ = df__[['from_id', '$node_id_9F94EAB3BA4046F396391C955D314E4B', 'src', 'dst', 'name', 'assoc_cond', 'icd11Code',
       'icd11Title', 'icd10Code', 'icd10Title',
        ]] 
df__.columns = ['from_id', 'to_id', 'src', 'dst', 'disgenet_name', 'assoc_cond', 'icd11Code',
       'icd11Title', 'icd10Code', 'icd10Title',
        ]

df__

,from_id,to_id,src,dst,disgenet_name,assoc_cond,icd11Code,icd11Title,icd10Code,icd10Title
0,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB00305,C0017601,Glaucoma,Ab externo surgery Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
1,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB00305,C0017601,Glaucoma,Ab externo surgery Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
2,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB00305,C0017601,Glaucoma,Ab externo surgery Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
3,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB00305,C0017601,Glaucoma,Ab externo surgery Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
4,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB00305,C0017601,Glaucoma,Ab externo surgery Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
...,...,...,...,...,...,...,...,...,...,...
4426102,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB01390,C0232567,Hypergastrinemia,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin
4426103,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB01390,C0232567,Hypergastrinemia,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin
4426104,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB01390,C0232567,Hypergastrinemia,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin
4426105,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB01390,C0232567,Hypergastrinemia,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin


In [7]:
df__ = df__.drop_duplicates()

In [8]:
# DUMP TO DISK
df__.to_csv("./ingest/drug_disease_edges.csv", sep="\t", index=False)

In [9]:
df__

,from_id,to_id,src,dst,disgenet_name,assoc_cond,icd11Code,icd11Title,icd10Code,icd10Title
0,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB00305,C0017601,Glaucoma,Ab externo surgery Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
1,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB00305,C0017601,Glaucoma,Ab externo surgery Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
2,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB00305,C0017601,Glaucoma,Ab externo surgery Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
3,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB00305,C0017601,Glaucoma,Ab externo surgery Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
4,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB00305,C0017601,Glaucoma,Ab externo surgery Glaucoma,9C61,Glaucoma,H40-H42,Glaucoma
...,...,...,...,...,...,...,...,...,...,...
4426033,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB01390,C0043515,Zollinger-Ellison syndrome,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin
4426034,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB01390,C0043515,Zollinger-Ellison syndrome,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin
4426101,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB01390,C0232567,Hypergastrinemia,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin
4426102,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""disease...",DB01390,C0232567,Hypergastrinemia,Zollinger-Ellison Syndrome,5A43.1,Zollinger-Ellison syndrome,E16.4,Abnormal secretion of gastrin
